<a href="https://colab.research.google.com/github/nyquist/aoe2de/blob/main/AOE2DE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Config Replay

In [100]:
#game_id = "358345344" #"357732557"

Get latest match ids

In [101]:
import requests
#profile_id = "18446743"
relic_id = "222475"
#match_type = 4
#game = "age2"

#profile_url = f"https://www.ageofempires.com/stats/?profileId={profile_id}&game={game}&matchType={match_type}"
profile_url = f"https://www.aoe2insights.com/user/{relic_id}/matches/"
response = requests.get(profile_url)
#print (response.status_code)
#print (response.headers)
import re
matches = re.finditer(r'href="/match/(\d+)/"', response.text)
games = []
for match in matches:
    found_game_id = match.group(1)
    games.append(found_game_id)
game_id = games[0]
print (game_id)
new_game = False
with open("last_game.txt", "r") as f:
  last_game = f.read()
  print ("Last Game",last_game)
  print ("Current Game", game_id)
  if last_game != game_id:
    new_game = True
with open("last_game.txt", "w") as f:
  f.write(game_id)





358591067
Last Game 358580260
Current Game 358591067


#Environment Setup

In [102]:
! [[ -d "aoe2de" ]] && (cd aoe2de && git pull) || git clone https://github.com/nyquist/aoe2de.git
! ls -la ./aoe2de/utils
! tail aoe2de/mtmExtractBuildingsfromJson.py

Already up to date.
total 12
drwxr-xr-x 2 root root 4096 Dec 15 19:30 .
drwxr-xr-x 5 root root 4096 Dec 15 20:11 ..
-rw-r--r-- 1 root root   22 Dec 15 19:30 test.py
        else:
            mtable = self.ecoTable
        mtable.set_style(MARKDOWN)
        return mtable.get_string()
      

if __name__ == '__main__':
    print ("Need to fix passing file_name. Try using it as a module")
    analyzer = BuildingsAnalyzer("AgeIIDE_Replay_358345344.aoe2record.json")
    print (analyzer.getTable("eco"))


In [103]:
import os

def list_files_with_dynamic_sizes(path="."):
    """Lists all files and their sizes with dynamic units (B, kB, MB, etc.) in the given directory.

    Args:
        path (str, optional): The directory path to list files from. Defaults to "." (current directory).
    """
    for entry in os.scandir(path):
        if entry.is_file():
            file_size = os.path.getsize(entry.path)
            unit_index = 0
            units = ["B", "kB", "MB", "GB", "TB"]
            while file_size >= 1024 and unit_index < len(units) - 1:
                file_size /= 1024
                unit_index += 1
            print(f"{entry.name}: {file_size:.2f} {units[unit_index]}")

# Call the function to list files and sizes with dynamic units in the current directory


# Download record

In [104]:
import requests

replay_url = f"https://aoe.ms/replay/?gameId={game_id}&profileId=18446743"  # Replace with the actual URL you copied

response = requests.get(replay_url)
response.raise_for_status()  # Raise an exception for bad responses
print (response.status_code)
content_disposition = response.headers['Content-Disposition']
import re
match = re.search(r'filename="?([^";]+)"?', content_disposition)
if match:
    file_name = match.group(1)
    print(f"Extracted filename: {file_name}")
    with open(file_name, "wb") as f:
      f.write(response.content)
else:
    print("Filename not found in Content-Disposition header.")


print(f"Replay file downloaded as {file_name}")
#list_files_with_dynamic_sizes()





200
Extracted filename: AgeIIDE_Replay_358591067.zip
Replay file downloaded as AgeIIDE_Replay_358591067.zip


### Unzip record file

In [105]:
zip_file_name = file_name
record_file_name = file_name.replace(".zip", ".aoe2record")

import zipfile
import os

try:
    with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
        zip_ref.extractall(".")  # Extract all files to the current directory
    print(f"Successfully extracted files from {zip_file_name}")

    # Find the extracted replay file (assuming it's an .mgz file)
    extracted_file_name = None
    for filename in os.listdir("."):
        print (filename)
        if filename == record_file_name:
            extracted_file_name = filename
            break

    if extracted_file_name:
        # Process the extracted replay file
        print ("Extracted file:", extracted_file_name)
    else:
        print("Error: Could not find extracted replay file (.mgz).")

except zipfile.BadZipFile:
    print(f"Error: {zip_file_name} is not a valid zip file.")
except FileNotFoundError:
    print(f"Error: {zip_file_name} not found.")
except RuntimeError as e:
    if "could not parse" in str(e):
        print(f"Error parsing the replay file: {e}")
        print("The replay file might be corrupted or in an unsupported format.")
        print("Please try re-downloading the replay file or using a different one.")
    else:
        raise  # Re-raise the exception if it's not related to parsing

Successfully extracted files from AgeIIDE_Replay_358591067.zip
.config
AgeIIDE_Replay_358567893.aoe2record
AgeIIDE_Replay_358567893.aoe2record.json
last_game.txt
AgeIIDE_Replay_358591067.zip
AgeIIDE_Replay_358591067.aoe2record
Extracted file: AgeIIDE_Replay_358591067.aoe2record


## Convert file to json

In [106]:
import json
! pip install mgz
from mgz.model import parse_match, serialize
f = extracted_file_name
outFile=f"{extracted_file_name}.json"
print (f)
with open(f, 'rb') as h:
    match = parse_match(h)
    # print(json.dumps(serialize(match), indent=2))
    json_object=json.dumps(serialize(match), indent=2)
    with open(outFile, "w") as o:
        o.write(json_object)

#list_files_with_dynamic_sizes()

AgeIIDE_Replay_358591067.aoe2record


# Analysis

In [107]:
with open(outFile, 'r') as json_file:
    json_data = json.load(json_file)

## Buildings

In [108]:
! tail aoe2de/mtmExtractBuildingsfromJson.py
! pip3 install prettytable

from aoe2de.mtmExtractBuildingsfromJson import BuildingsAnalyzer

all_buildings = BuildingsAnalyzer(outFile)

        else:
            mtable = self.ecoTable
        mtable.set_style(MARKDOWN)
        return mtable.get_string()
      

if __name__ == '__main__':
    print ("Need to fix passing file_name. Try using it as a module")
    analyzer = BuildingsAnalyzer("AgeIIDE_Replay_358345344.aoe2record.json")
    print (analyzer.getTable("eco"))
+----------------+----------------+---------+---------+-------------+---------+----+------+----------------+
|    Cladire     | email_to_mihai | imporer | nyquist | m.yesiltepe | gabisan | RA | risk | flowfranking22 |
+----------------+----------------+---------+---------+-------------+---------+----+------+----------------+
|   Blacksmith   |       1        |    1    |    1    |      1      |    1    | 1  |  2   |       1        |
|    Barracks    |       6        |    1    |    3    |      11     |    7    | 0  |  0   |       1        |
|     Stable     |       8        |    1    |    4    |      0      |    5    | 0  |  5   |       11       |
| Archer

In [109]:
def discordMessage(content):
  from google.colab import userdata
  userdata.get('webhook')

  import requests

  webhook_url = userdata.get('webhook')
  data = {"content": content}

  response = requests.post(webhook_url, json=data)

  if response.status_code == 204:
      print("Message sent successfully!")
  else:
      print(f"Failed to send message: {response.status_code}")
if new_game:
  discordMessage(f"Last game {game_id}")
  discordMessage(f"```{all_buildings.getTable('eco')}```")
  discordMessage(f"```{all_buildings.getTable('mil')}```")
else:
  print ('No new games')
#type(all_buildings)



Message sent successfully!
Message sent successfully!
Message sent successfully!


## Army